## 5b Univariate und bivariate Statistik

<div class="alert alert-info"> In dieser Lerneinheit beschäftigen wir uns mit statistischen Berechnungen in Python. Hierfür sehen wir uns zunächst noch einmal an, wie ein Datensatz in Python aufgebaut ist. Im Anschluss daran, berechnen wir mithilfe von <b>scipy.stats</b> und <b>statistics</b> Lage-, Streuungs- und Korrelationsmaße. Außerdem sehen wir uns spezielle Verteilungen an und lernen, wie statistische Tests in Python umzusetzen sind. Aufgebaut ist die Lerneinheit wie folgt: 
    <ol>
        <li>Häufigkeitsverteilung und empirische Verteilungsfunktion</li>
        <li>Lage und Streuung</li>
        <li>Zusammenhangsmaße</li>
        <ol>
            <li>Zwei nominale Variablen</li>
            <li>Zwei ordinale Variablen</li>
            <li>Zwei metrische Variablen</li>
        </ol>
        <li>Spezielle Verteilungen</li>
        <li>Statistische Tests</li>
    </ol>

Nach dieser Lerneinheit können Sie sich unterschiedliche Lage- und Streuungsmaße ausgeben lassen.Zusammenhangsmaße in Python berechnen und deren Ergebnis interpretieren. Sie können (Pseudo-)Zufallszahlen generieren. Sie können statistische Tests auswählen und anwenden und deren Ergebnisse in verbalisierter Form interpretieren.</div>

Pakete, die noch installiert werden müssen: 
* scipy
* matplotlib
* scikit-learn

#### Pakete laden

In [ ]:
import csv

import pandas as pd
import numpy as np
import scipy.stats as stats
import statistics

In [ ]:
df = pd.read_csv("..\_Daten\Datensatz_Herzinfarkt.csv", sep=",")
df.head() 

In [ ]:
print(df.Geschlecht.unique())
print(df.Schmerztyp.unique())
print(df.Blutzucker.unique())
print(df.Herzinfarkt.unique())
print(df.Schmerzintensität.unique())

In [ ]:
df.isnull().sum() # Keine Missings

In [ ]:
df.dtypes

In [ ]:
# Schmerzintensität als ordinales Merkmal
cat = ["Keine Schmerzen", "Kaum Schmerzen", "Mittel", "Starke Schmerzen", "Sehr starke Schmerzen"]
df["Schmerzintensität"] = pd.Categorical(df["Schmerzintensität"], categories=cat, ordered=True)

In [ ]:
# Blutzucker als nominales Merkmal
df["Blutzucker"] = df["Blutzucker"].astype(object)

In [ ]:
df.dtypes

## 1 Häufigkeitsverteilung und empirische Verteilungsfunktion

**Nominale Merkmale**

In [ ]:
df["Schmerztyp"].value_counts() # absolute Häufigkeiten

In [ ]:
df["Schmerztyp"].value_counts(sort = False)

In [ ]:
df["Schmerztyp"].value_counts()/len(df) # relative Häufigkeiten

In [ ]:
pd.crosstab(df["Herzinfarkt"], df["Schmerztyp"]) # Kontingenztabelle

In [ ]:
pd.crosstab(df["Herzinfarkt"], df["Schmerztyp"], margins = True) # Kontingenztabelle

**Ordinale Merkmale**

In [ ]:
df.Schmerzintensität.value_counts(sort = False) # sort = Datenwerte nach Häufigkeit sortieren

In [ ]:
df["Schmerzintensität"].value_counts(sort = False)/len(df)

In [ ]:
np.cumsum(df["Schmerzintensität"].value_counts(sort = False)/len(df))

## 2 Lage und Streuung

#### Mittelwert

In [ ]:
# Option 1: Pandas
df.mean(numeric_only=True)

In [ ]:
# Option 2: NumPy
np.mean(df["Alter"])

#### Median

In [ ]:
# Option 1: Pandas
df.median(numeric_only=True)

In [ ]:
# Option 2: NumPy
np.median(df["Alter"])

#### Maximum und Minimum

In [ ]:
df.max(numeric_only=True)

In [ ]:
df.min(numeric_only=True)

#### Quantile

In [ ]:
# Option 1: Scipy
stats.scoreatpercentile(df["Alter"], per=50)

In [ ]:
stats.scoreatpercentile(df["Alter"], (25, 50, 75))

In [ ]:
# Option 2: NumPy
np.quantile(df["Alter"], .5)

In [ ]:
np.quantile(df["Alter"], (.25, .5, .75))

#### Varianz und Stichprobenvarianz

In [ ]:
# Option 1: Pandas
df.var(numeric_only=True) # Stichprobenvarianz

In [ ]:
# Option 2: Statistics
print("Stichprobenvarianz: {}".format(statistics.variance(df["Alter"]))) # Stichprobenvarianz
print("Varianz: {}".format(statistics.pvariance(df["Alter"]))) # Varianz

In [ ]:
# Option 3: NumPy
print("Stichprobenvarianz: {}".format(np.var(df["Alter"], ddof=1))) # Stichprobenvarianz
print("Varianz: {}".format(np.var(df["Alter"]))) # Varianz

#### (Empirische) Standardabweichung

In [ ]:
# Pandas
df.std(numeric_only=True) # Emp. Standardabweichung

In [ ]:
# Option 2: Statistics
print("Emp. Standardabweichung: {}".format(statistics.stdev(df["Alter"]))) # Emp. Standardabweichung
print("Standardabweichung: {}".format(statistics.pstdev(df["Alter"]))) # Standardabweichung

In [ ]:
# Option 3:NumPy
print("Emp. Standardabweichung: {}".format(np.std(df["Alter"], ddof=1))) # Emp. Standardabweichung
print("Standardabweichung: {}".format(np.std(df["Alter"]))) # Standardabweichung

<div class="alert alert-warning"><h4> Aufgabe 1: Lage und Streuung

a) Laden Sie den `iris`-Datensatz als `df_iris` aus dem `seaborn`-Paket.<br>

b) Geben Sie für alle metrischen Variablen ein geeignetes Lokalisationsmaß aus.  

c) Geben Sie für jede Variable des gesamten Datensatz das Maximum und Minimum aus.

e) Bestimmen Sie die relative Häufigkeitsverteilung der Variablen `species`. 

## 3 Zusammenhangsmaße

### 3.1 Zwei nominale Variablen

Cramérs V: $\sqrt(\frac{\chi^2}{nm})$ mit
* $\chi^2$ als Teststatistik des $\chi^2$-Tests,
* $n$ die Stichprobengröße und
* $m$ das Minimum von **Anzahl an Spalten-1** und **Anzahl an Zeilen -1**

In [ ]:
ct = pd.crosstab(df["Herzinfarkt"], df["Schmerztyp"]) # Kontingenztabelle
ct

In [ ]:
help(stats.chi2_contingency)

In [ ]:
stats.chi2_contingency(ct)

In [ ]:
X2 = stats.chi2_contingency(ct)[0]

In [ ]:
n = len(df)

In [ ]:
m = min(ct.shape)-1

In [ ]:
ct.shape

In [ ]:
min(ct.shape)

In [ ]:
np.sqrt(X2/(n*m))

### 3.2 Zwei ordinale Variablen

* Kendall'sche Tau 
* Spearman'scher Rangkorrelationskoeffizient

In [ ]:
# Neues ordinales Merkmal - Kapitel 6.4 Spezielle Verteilungen
import random
random.seed(123)
rand = random.choices([1, 2, 3], k=len(df))
print(rand)

In [ ]:
help(stats.kendalltau)

In [ ]:
# Kendall'sche Tau
stats.kendalltau(rand, df.Schmerzintensität)

In [ ]:
corr, p = stats.kendalltau(rand, df.Schmerzintensität)
print(corr, p)

In [ ]:
# Spearman'scher Rangkorrelationskoeffizient
stats.spearmanr(rand, df.Schmerzintensität)

### 3.3 Zwei metrische Variablen

* Kovarianz
* Bravais-Pearson-Korrelation
* Lineare Regression

In [ ]:
# Visualisierung - mehr dazu in Lerneinheit 6 Graphische Darstellung
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.scatter(df.iloc[:,3], df.iloc[:,4], )
plt.xlabel(df.columns[3])
plt.ylabel(df.columns[4])
plt.show()

**Kovarianz**

In [ ]:
df.iloc[:,4].cov(df.iloc[:,3])

**Bravais-Pearson-Korrelation**

In [ ]:
# Option 1: NumPy
r1 = np.corrcoef(df.iloc[:,4], df.iloc[:,3])
print(r1)
print("=========================") # Nur zur besseren Übersichtlichkeit
print(np.round(r1[0, 1],3))

In [ ]:
# Option 2: SciPy
r2, p2 = stats.pearsonr(df.iloc[:,4], df.iloc[:,3]) # Korrelationskoeffizient und p-Wert
print(r2, p2)

**Lineare Regression**

In [ ]:
# scipy
stats.linregress(df.iloc[:,4], df.iloc[:,3])

In [ ]:
# sklearn
from sklearn.linear_model import LinearRegression
model = LinearRegression()

x = np.array(df.iloc[:,4]).reshape(-1,1)
y = np.array(df.iloc[:,3])
model.fit(x, y)

In [ ]:
print("Intercept:", model.intercept_)
print("Steigung:", model.coef_)
print("R²:", model.score(x, y))

## 4 Spezielle Verteilungen

#### (Pseudo-)Zufallszahlen mit NumPy

Mithilfe des `random`-Moduls von NumPy lassen sich einfach und schnell Pseudozufallszahlen generieren. Durch das Setzen eines *Seeds* lassen sich die gleichen Zufallszahlen erneut generieren, sodass die Berechnungen nachvollziehbar sind. Die Befehle sind dabei wie folgt aufgebaut: <br>
np.random.**verteilung**()

| Argument np.random._    | Wert                | Beschreibung                            |
|:------------|:--------------------------------|:-----------------------------------------|
| _binomial()  |  n, p                          | Binomialverteilung *B(n, p)*             |
| _hypergeometric()   | ngood, nbad, nsample    | Hypergeometrische Verteilung *Hyp(N, M, n)* |
| _poisson()       |                     lam    | Poissonverteilung *P($\lambda$)*               |
| _normal()   |      loc, scale                 | Normalverteilung $N(\mu, \sigma^2)$ |
| _uniform()    | low, high                     | Gleichverteilung *U[a, b]*                    |
| _exponential()   | scale                      | Exponentialverteilung $Exp(\lambda)$ |
|_t()              |   df                       | *t*-Verteilung $t(n)$ |
|_chisquare()      |  df                        | $\chi^2$-Verteilung $\chi^2(n)$ |
|_f()             |   dfnum, dfden              | *F*-Verteilung $F(m, n)$

#### Beispiel: Diskret

In [ ]:
# Binomialverteilung: B(n, p)
np.random.binomial(n=5, p=0.25, size=10)

In [ ]:
# Seed
np.random.seed(345)
np.random.binomial(n=1, p=0.5, size=10)

In [ ]:
# Poisson
np.random.poisson(lam=3, size=10)

#### Stetig

In [ ]:
# Standardnormalverteilung
np.random.normal(size=10)

In [ ]:
# Normalverteilung: N(µ, σ^2)
np.random.normal(loc=100, scale=30, size=10)

#### SciPy

| Funktion scipy.stats._.%|Argumente          |Beschreibung                             |
|:------------------------|:------------------|:----------------------------------------|
| %rvs()                  |size, random_state |(Pseudo-)Zufallsvariablen                |
| %pmf()                  | k                 |Wahrscheinlichkeitsfunktion              |
| %pdf()                  | x                 |Dichtefunktion                          |
| %cdf()                  | x / k             |Verteilungsfunktion                      |

| Funktion scipy.stats._.%| Argumente                | Beschreibung                             |
|:------------------------|:-------------------------|:-----------------------------------------|
| _binom                  | n, p                     | Binomialverteilung $B(n, p)$             |
| _hypergeom              | M, n, N                  | Hypergeometrische Vtlg. $Hyp(N, M, n)$   |
| _poisson                | mu = $\lambda$           | Poissonverteilung $P(\lambda)$           |
| _norm                   |loc=$\mu$, scale=$\sigma$ | Normalverteilung $N(\mu, \sigma^2)$      |
| _uniform                |loc = $a$, scale=$b$      | Gleichverteilung $U[a, b]$               |
| _expon                  |scale=$1/\lambda$         | Exponentialverteilung $Exp(\lambda)$     |
| _t                      |df                        | *t*-Verteilung $t(df)$                   |
| _f                      |dfn = m, dfd = n          | *F*-Verteilung $F(m, n)$                 |
| _chi2                   |df = n                    | $\chi^2$-Verteilung $\chi^2(n)$          |

In [ ]:
# Beispiel 1: Binomialverteilung (mehr dazu in Kapitel 7!)
data = stats.binom.rvs(n = 10, p = 0.25, size = 30, random_state = 1234)
plt.hist(data, bins=5, density = True, label = "PMF")
plt.hist(data, bins=5, density = True, cumulative = True, label ="CDF", histtype = "step")
plt.xlabel("X")
plt.ylabel("Probability")
plt.xticks(np.arange(0,6))
plt.title("PMF and CDF: Bin(n = 10, p = 0.25)")
plt.legend(loc = "upper left")
plt.show()

In [ ]:
stats.binom.cdf(k = 3, n = 10, p = 0.25)

In [ ]:
stats.binom.pmf(k = 3, n = 10, p = 0.25)

In [ ]:
stats.binom.rvs(n = 10, p = 0.25, size = 5, random_state=1234)

In [ ]:
# Beispiel 2: Normalverteilung
data_norm = stats.norm.rvs(loc = 0, scale = 1, size = 50, random_state = 123)
data_sorted = np.sort(data_norm)
x = stats.norm.pdf(data_sorted)
y = stats.norm.cdf(data_sorted)

plt.plot(data_sorted, x, label = "pdf")
plt.plot(data_sorted, y, label = "cdf")
plt.legend()
plt.title("PDF and CDF: N(0,1)")
plt.xlim([-3, 3])
plt.show()

In [ ]:
stats.norm.cdf(x = 0, loc = 0, scale = 1)

In [ ]:
stats.norm.pdf(x = 0, loc = 0, scale = 1)

In [ ]:
stats.norm.rvs(loc = 0, scale = 1, size = 5, random_state=5678)

## 5 Statistische Tests

Statistische Tests sind Verfahren zur Überprüfung von Annahmen (Hypothesen) über die unbekannte Verteilung oder unbekannte Parameterwerte von Zufallsvariablen in der Grundgesamtheit unter Verwendung der Ergebnisse von Zufallsstichproben.<br>
**Hypothesenpaar:**
* H<sub>0</sub>: Nullhypothese - die statistische Formulierung der zu prüfenden Annahme
* H<sub>1</sub>: Alternativhypothese - die der H<sub>0</sub> entgegengestellte Hypothese

**Teststatistik**: Stichprobenfunktion der Stichprobenvariablen mit bekannter (approx.) Verteilung <br>
**Testentscheidung**: Die H<sub>0</sub> wird verworfen, wenn ein signifikanter Widerspruch aufgrund des Stichprobenergebnisses besteht:
* Ablehnungbereich: H<sub>0</sub> wird abgelehnt, falls der Wert der Teststatistik in den Ablehnungsbereich zur H<sub>0</sub> fällt.
* *p*-Wert: Wert des Signifikanzniveaus, für den der Wert der Teststatistik auf den Rand des Ablehnungsbereichs fällt; Wahrscheinlichkeit, mit der sich unter Gültigkeit der H<sub>0</sub> der aus der Stichprobe resultierende Wert der Teststatistik oder ein in Richtung der H<sub>1</sub> noch extremerer Wert ergibt.

**Achtung**: H<sub>0</sub> wird abgeleht, falls *p*-Wert < $\alpha$<br>

| Funktion       | Beschreibung                                     |
|:---------------|:-------------------------------------------------|
| t.test()       | *t*-Test (1- und 2-STP.) zum Mittelwertsvergleich|
| levene()       | Levene-Test auf Varianzhomogenität               |
| shapiro.test() | Shapiro-Wilk Test auf Normalverteilung           |
| chisq.test()   | Anpassungs- bzw. Unabhängigkeitstest             |
| cor.test()     | Korrelationstest                                 |

#### 1- und 2-Stichproben *t*-Test

Der Einstichproben *t*-Test prüft anhand des Mittelwertes einer Stichprobe, ob sich der Mittelwert einer Grundgesamtheit von einem vorgegeben Sollwert unterscheidet. <br>
Es können drei verschiedene Hypothesenpaare formuliert werden: 
1. H<sub>0</sub>: $\mu$ = $\mu$<sub>0</sub> vs. H<sub>1</sub>: $\mu$ ≠ $\mu$<sub>0</sub> (zweiseitig)
2. H<sub>0</sub>: $\mu$ ≤ $\mu$<sub>0</sub> vs. H<sub>1</sub>: $\mu$ > $\mu$<sub>0</sub> (rechtsseitig)
3. H<sub>0</sub>: $\mu$ ≥ $\mu$<sub>0</sub> vs. H<sub>1</sub>: $\mu$ < $\mu$<sub>0</sub> (linksseitig)

#### Einstichproben *t*-Test
scipy.stats.ttest_1samp(a, popmean, nan_policy="omit", alternative="two-sided")
* **a**: Beobachtungen der Stichprobe
* **popmean**: Erwarteter Wert der Nullhypothese (float)
* **nan_policy**: Umgang mit fehlenden Werten: *propagate* (default: Gibt Nan zurück), *raise* (Fehlermeldung) oder *omit* (Ignorieren)
* **alternative**: Alternativhypothese: *two-sided* (default), *greater* oder *less*

In [ ]:
# H_0: Blutdruck = 140
stats.ttest_1samp(df["Blutdruck"], 140, alternative="two-sided")
# da pvalue < 0.05 --> H0 ablehnen

In [ ]:
# H_0: Blutdruck <= 140
stats.ttest_1samp(df["Blutdruck"], 140, alternative="greater")
# da pvalue > 0.05 --> H0 nicht ablehnen

In [ ]:
# H_0: Blutdruck >= 140
stats.ttest_1samp(df["Blutdruck"], 140, alternative="less")
# da pvalue < 0.05 --> H0 ablehnen

#### Zweistichproben *t*-Test
scipy.stats.ttest_ind(a, b, equal_var=True, nan_policy="omit", alternative="two-sided")
* **a** und **b**: Beobachtungen der zwei unabhängigen Stichproben
* **equal_var**: Wenn *True* (default), dann Annahme identischer Varianzen. Wenn *False*, dann Welch's *t*-Test

In [ ]:
# 2 Gruppen: Herzinfarkt ja oder nein
df_group = df.groupby(df.Herzinfarkt)

In [ ]:
# H_0: kein Unterschied zwischen den Gruppen
stats.ttest_ind(df_group.get_group("Herzinfarkt")["Cholesterin"], df_group.get_group("Kein Herzinfarkt")["Cholesterin"])
# da pvalue > 0.05 --> H0 nicht ablehnen

#### Test auf Normalverteilung

Überprüft die Hypothese, dass die zugrunde liegende Grundgesamtheit einer Stichprobe normalverteilt ist: 
* $H_0: F = F_0$ mit $F_0$ als Normalverteilung
* $H_1: F \neq F_0$ <br>

In [ ]:
# H_0: Blutdruck normalverteilt
stats.shapiro(df.Blutdruck)
# pvalue > 0.05 --> H0 nicht ablehnen

#### Test auf Varianzhomogenität

Überprüft die Nullhypothese, dass alle Gruppenvarianzen gleich sind:
* $H_0: \sigma^2_1 = \sigma^2_2 = ... = \sigma^2_k$
* $H_1: \sigma^2_i \neq \sigma^2_j$ für mindestens ein Gruppenpaar $i, j$ mit $i \neq j$

In [ ]:
# H_0: Varianzen zwischen allen Gruppen gleich
stats.levene(df_group.get_group("Herzinfarkt")["Cholesterin"], df_group.get_group("Kein Herzinfarkt")["Cholesterin"])
# p > 0.05 --> H0 nicht ablehnen

#### Chi-Quadrat Anpassungstest

Überprüft die Hypothese, dass die zugrunde liegende Grundgesamtheit einer Stichprobe einer bestimmten Verteilungsklasse folgt: 
* $H_0: F = F_0$ mit $F_0$ als hypothetische Verteilungsfunktion
* $H_1: F \neq F_0$ <br>

In [ ]:
# Schmerztyp gleichverteilt: 4 Schmerztypen
df_counts = df.Schmerztyp.value_counts()
erwartet = [len(df)/4, len(df)/4, len(df)/4, len(df)/4]

stats.chisquare(f_obs = df_counts, f_exp=erwartet)
# p < 0.05 --> H0 ablehnen

<div class="alert alert-warning"><h4> Aufgabe 2: Pseudozufallszahlen

a) Ziehen Sie eine Stichprobe der Größe *n* = 99 aus einer Normalverteilung mit $\mu$ = 5 und $\sigma^2$ = 4 (set.seed = 1).

b) Nehmen Sie an $X \sim Poi(\lambda = 2)$. Geben Sie folgende Werte an: 
* Verteilungsfunktion $F(x)$ an der Stelle 1 an. 
* Wahrscheinlichkeitsfunktion $f(x)$ an der Stelle 1. 

c) Nehmen Sie die Zufallsvariable $Y$ sei standardnormalverteilt. Geben Sie den Wert der Dichte- und der Verteilungsfunktion an der Stelle 0 an.